<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/classify_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run images through image type classification pipeline
---
*Last Updated 31 July 2022*  
-Runs in Python 3 with Tensorflow 2.0-   

Use trained image classification model to add tags for image type (map, phylogeny, illustration, herbarium sheet) to EOL images.

Models were trained in [image_type_train.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/image_type_train.ipynb). Confidence threshold for the best trained model was selected in [inspect_train_results.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/inspect_train_results.ipynb). 

In post-processing, an additional tag is used to filter images because the model did not learn to predict phylogenies or illustrations very well. Using "cartoonization" in  [cartoonize_images.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/cartoonify_images.ipynb), image properties (Manhattan norm per pixel) are leveraged to classify images as photographic or non-photographic. Tags with matching cartoonized and image type tags above chosen thresholds are kept. 

Finally, display tagging results on images to verify behavior is as expected.

***Models were trained in Python 2 and TF 1 in October 2020: MobileNet SSD v2 was trained for 3 hours to 30 epochs with Batch Size=16, Lr=0.00001, Dropout=0.3, epsilon=1e-7, Adam optimizer. Final validation accuracy = 0.90.***

Notes:     
* Run code blocks by pressing play button in brackets on left
* Before you you start: change the runtime to "GPU" with "High RAM"
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param')

## Installs & Imports
---

In [ ]:
#@title Choose where to save results
import os

# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]

# Mount google drive to export image tagging file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type in the path to your project wd in form field on right
basewd = "/content/drive/MyDrive/train" #@param ["/content/drive/MyDrive/train"] {allow-input: true}

# Make folder for image tags within base wd
cwd = basewd + '/results/'
if not os.path.exists(cwd):
    os.makedirs(cwd)

print("Saving results ", save)

In [ ]:
#@title Import libraries

# For downloading and displaying images
from PIL import Image
import cv2
import imageio
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# For working with data
import numpy as np
import pandas as pd
from os import path
import csv
import itertools
from scipy.linalg import norm
from scipy import sum, average
# So URL's don't get truncated in display
pd.set_option('display.max_colwidth',1000)
pd.options.display.max_columns = None

# For measuring inference time
import time

# For image classification
import tensorflow as tf
print('\nTensorflow Version: %s' % tf.__version__)

# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

### Prepare classification functions and settings

In [ ]:
#@title Define functions

# To read in EOL formatted data files
def read_datafile(fpath, sep="\t", header=0, disp_head=True, lineterminator='\n', encoding='latin1'):
    try:
        df = pd.read_csv(fpath, sep=sep, header=header, lineterminator=lineterminator, encoding=encoding)
        if disp_head:
          print("Data header: \n", df.head())
    except FileNotFoundError as e:
        raise Exception("File not found: Enter the path to your file in form field and re-run").with_traceback(e.__traceback__)
    
    return df

# Define start and stop indices in EOL bundle for running inference   
def set_start_stop(df):
    # To test with a tiny subset, use 5 random bundle images
    if "tiny subset" in run:
        start=np.random.choice(a=len(df), size=1)[0]
        stop=start+5
    # To run inference on 4 batches of 5k images each
    elif "_a." in outfpath: # batch a is from 0-5000
        start=0
        stop=5000
    elif "_b." in outfpath: # batch b is from 5000-1000
        start=5000
        stop=10000
    elif "_c." in outfpath: # batch c is from 10000-15000
        start=10000
        stop=15000
    elif "_d." in outfpath: # batch d is from 15000-20000
        start=15000
        stop=20000

    return start, stop

# Load in image from URL
# Modified from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb#scrollTo=JhVecdzJTsKE
def image_from_url(url, fn):
    # Formatted for classification
    f = tf.keras.utils.get_file(fn, url) # Filename doesn't matter
    disp_img = tf.keras.preprocessing.image.load_img(f) # For display
    img_cv = np.array(disp_img) # For working with cv2 lib
    image = tf.keras.preprocessing.image.load_img(f, target_size=[pixels, pixels])
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(
        image[tf.newaxis,...])
    
    return image, disp_img, img_cv

# To cartoonize an image
def cartoonize(img_cv):
    # Add edges
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY) 
    gray = cv2.medianBlur(gray, 5) 
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,  
                                         cv2.THRESH_BINARY, 9, 9)  
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    # Bilateral filter 
    color = cv2.bilateralFilter(img_cv, 9, 250, 250) 
    img2 = cv2.bitwise_and(color, edges)

    return img2

# Calculate differences between original and cartoonized image
def calc_img_diffs(img, img2):
    # Convert both images from RGB to HSV
    HSV_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    HSV_img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2HSV)
    # Fnd the difference for H of HSV values of the images
    diff = HSV_img[:,:,0]-HSV_img2[:,:,0]
    mnorm = sum(abs(diff))  # Manhattan norm
    mnorm_pp = mnorm/HSV_img.size # per pixel
    znorm = norm(diff.ravel(), 0)  # Zero norm
    znorm_pp = znorm*1.0/HSV_img2.size # per pixel

    return mnorm_pp, mnorm, znorm_pp, znorm


# Load saved model from directory
def load_saved_model(saved_models_dir, TRAIN_SESS_NUM, module_selection):
    # Load trained model from path
    saved_model_path = saved_models_dir + TRAIN_SESS_NUM
    model = tf.keras.models.load_model(saved_model_path)
    # Get name and image size for model type
    handle_base, pixels = module_selection

    return model, pixels, handle_base

# Get info from predictions to display on images
def get_predict_info(predictions, url, i, stop, start):
    # Get info from predictions
    label_num = np.argmax(predictions[0], axis=-1)
    conf = predictions[0][label_num]
    im_class = dataset_labels[label_num]
    # Display progress message after each image
    print("Completed for {}, {} of {} files".format(url, i+1, format(stop-start, '.0f')))
    
    return label_num, conf, im_class

# Set filename for saving classification results
def set_outpath(tags_file):
    outpath = basewd + '/results/' + tags_file + '.tsv'
    print("\nSaving results to: \n", outpath)

    return outpath

# Export results
def export_results(df, url, mnorm_pp, det_imclass, conf):
    # Define variables for export
    if 'ancestry' in df.columns:
        ancestry = df['ancestry'][i]
    else:
        ancestry = "NA"
    identifier = df['identifier'][i]
    dataObjectVersionID = df['dataObjectVersionID'][i] 
    # Write row with results for each image
    results = [url, identifier, dataObjectVersionID, ancestry, mnorm_pp, 
               det_imclass, conf]
    with open(outfpath, 'a') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(results)

In [ ]:
#@title Choose saved model parameters (if using EOL model, defaults are already selected)

# Use EOL pre-trained model for object detection?
use_EOL_model = True #@param {type: "boolean"}

# Get info about trained classification model
def get_model_info(use_EOL_model):
    # Use EOL pre-trained model
    if use_EOL_model:
        # Model metadata
        module_selection =('mobilenet_v2_1.0_224', 224)
        dataset_labels = ["map", "phylo", "illus", "herb", "null"]
        TRAIN_SESS_NUM = '13'
        saved_models_dir = basewd + '/saved_models/'
        # If running for the first time, download model
        if not os.path.exists(saved_models_dir):
            # Make folder for trained model
            os.makedirs(saved_models_dir)
            %cd $saved_models_dir
            os.makedirs(TRAIN_SESS_NUM)
            # Download saved model files for Run 13 - MobileNet SSD v2
            !gdown --id 1Fr1x5ZLXdd-DBZ7yRWx691orbtXh9lW1
            !unzip 13.zip -d .
            !mv -v content/drive/'My Drive'/summer20/classification/image_type/saved_models/13/* 13
            !rm -r content
            !rm -r 13.zip
            %cd ../
            print("\nSuccessfully downloaded pre-trained EOL model to: \n", (saved_models_dir + TRAIN_SESS_NUM))
    
    # Use your own trained model
    elif not use_EOL_model:
        # Change values to match your trained model
        module_selection = ("mobilenet_v2_1.0_224", 224) #@param ["(\"mobilenet_v2_1.0_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
        dataset_labels = ["map", "phylo", "illus", "herb", "null"] #@param ["[\"map\", \"phylo\", \"illus\", \"herb\", \"null\"]"] {type:"raw", allow-input: true}
        saved_models_dir = "train/saved_models/" #@param ["train/saved_models/"] {allow-input: true}
TRAIN_SESS_NUM = "13" #@param ["13"] {allow-input: true}

    return module_selection, dataset_labels, saved_models_dir, TRAIN_SESS_NUM

# Load saved model
module_selection, dataset_labels, saved_models_dir, TRAIN_SESS_NUM = get_model_info(use_EOL_model)
model, pixels, handle_base = load_saved_model(saved_models_dir, TRAIN_SESS_NUM, module_selection)

### Generate tags: Run inference on EOL images & save results for tagging - Run 4X for batches A-D
Use 20K EOL image bundle to classify image type as map, phylogeny, illustration, or herbarium sheet. Results are saved to [tags_file].tsv. Run this section 4 times (to make batches A-D) of 5K images each to incrementally save in case of Colab timeouts.

In [ ]:
#@title Enter EOL image bundle and choose inference settings. Change **tags_file** for each batch A-D
%cd $cwd

# Load in EOL image bundle
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Squamata_20K_breakdown_000001.txt" #@param ["https://editors.eol.org/other_files/bundle_images/files/images_for_Squamata_20K_breakdown_000001.txt", "https://editors.eol.org/other_files/bundle_images/files/images_for_Coleoptera_20K_breakdown_000001.txt", "https://editors.eol.org/other_files/bundle_images/files/images_for_Anura_20K_breakdown_000001.txt", "https://editors.eol.org/other_files/bundle_images/files/images_for_Carnivora_20K_breakdown_000001.txt"] {allow-input: true}
df = read_datafile(bundle, sep='\t', header=0, disp_head=False)

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for all images"]

# Take 5k subset of bundle for running inference
# Change filename for each batch
tags_file = "image_type_tags_tf2_d" #@param ["image_type_tags_tf2_a", "image_type_tags_tf2_b", "image_type_tags_tf2_c", "image_type_tags_tf2_d"] {allow-input: true}
outfpath = set_outpath(tags_file)

# Write header row of tagging file
if not os.path.isfile(outfpath): 
    with open(outfpath, 'a') as out_file:
              tsv_writer = csv.writer(out_file, delimiter='\t')
              tsv_writer.writerow(["eolMediaURL", "identifier", 
                                   "dataObjectVersionID", "ancestry", \
                                   "mnorm_pp", "imclass", "conf"])

In [ ]:
#@title Run inference and cartoonify images as photographic/non-photographic
start, stop = set_start_stop(df)
for i, row in enumerate(df.iloc[start:stop].iterrows()):
    try:       
        # Read in image from url
        url = df['eolMediaURL'][i]
        fn = str(i) + '.jpg'
        img, disp_img, img_cv = image_from_url(url, fn)

        # Cartoonization
        img2 = cartoonize(img_cv)
        # Calculate differences between original and cartoonized image
        mnorm_pp, _, _, _ = calc_img_diffs(img_cv, img2)

        # Image classification
        start_time = time.time() # Record inference time
        predictions = model.predict(img, batch_size=1)
        label_num, conf, det_imclass = get_predict_info(predictions, url, i, stop, start)
        end_time = time.time()
        print("Inference time: {} sec".format(format(end_time-start_time, '.2f')))

        # Export cartoonization results to tsv
        export_results(df, url, mnorm_pp, det_imclass, conf)

    except:
        print('Check if URL from {} is valid'.format(url))

print("\n\n~~~\nInference complete! Run these steps for remaining batches A-D before proceeding.\n~~~")

## Post-process classification predictions using threshold values
---
Use Manhattan norm values for classifying images as photographic or non-photographic (Manhattan norm per pixel threshold < 2). Then, use MobileNet SSD v2 confidence values for classifying image type (confidence threshold > 1.6). When tag types do not match or are outside of thresholds, they are discarded.

*Example of tag types matching: Phylogeny, map, and illustration are all non-photographic (cartoons). If an image is classified as photographic and any of those classes, the tags will be discarded because it is likely a misidentification.*

### Prepare post-processing functions and settings

In [ ]:
#@title Adjust Manhattan norm and confidence threshold parameters (or use EOL defaults)

# Are images botanical? 
botanical = False #@param {type:"boolean"}

# More strict primary cartoonization and confidence thresholds
mnorm_thresh = 2 #@param ["2"] {type:"raw", allow-input: true}
conf_thresh = 1.6 #@param ["1.6"] {type:"raw", allow-input: true}

# Less strict secondary cartoonization and confidence thresholds
mnorm_thresh2 = 15 #@param ["15"] {type:"raw", allow-input: true}
conf_thresh2 = 0.05 #@param ["0.05"] {type:"raw", allow-input: true}
 
# Intermediate cartoonization threshold for herbarium sheets (less colors than photographic image, more than illustration)
mnorm_herb = 20 #@param ["20"] {type:"raw", allow-input: true}

# Define functions

# Combine tagging files for batches A-D
def combine_tag_files(tags_fpath):
    # Combine tag files for batches A-D
    fpath =  os.path.splitext(tags_fpath)[0]
    base = fpath.rsplit('_',1)[0] + '_'
    exts = ['a.tsv', 'b.tsv', 'c.tsv', 'd.tsv'] 
    all_filenames = [base + e for e in exts]
    df = pd.concat([pd.read_csv(f, sep='\t', header=0, na_filter = False) for f in all_filenames], ignore_index=True)
    # Choose desired columns for tagging
    df[['mnorm_pp', 'conf']] = df[['mnorm_pp', 'conf']].apply(pd.to_numeric)
    df.rename(columns={'conf': 'confidence', 'imclass': 'tag'}, inplace=True)
    # Make empty columns for final tag types
    df['tag_cartoon'] = np.nan
    df['tag_imtype'] = np.nan
    df['problematic'] = np.nan

    return df, base

# Filter by thresholds
def filter_by_thresholds(df, cartoonization, confidence):
    # Filter by cartoonization threshold for photo or non-photo
    idx_tokeep = df.index[df.mnorm_pp <= mnorm_thresh]
    idx_todiscard = df.index.difference(idx_tokeep)
    df.loc[idx_tokeep, 'tag_cartoon'] = 'non-photo'
    df.loc[idx_todiscard, 'tag_cartoon'] = 'photo'

    # Filter by classification confidence threshold
    idx_tokeep = df.index[df.confidence > conf_thresh]
    idx_todiscard = df.index.difference(idx_tokeep)
    df.loc[idx_tokeep, 'tag_imtype'] = df.loc[idx_tokeep, 'tag']
    df.loc[idx_todiscard, 'tag_imtype'] = 'null'

    return df

# Add 'problematic' tags to classes model didn't learn well
def add_problematic_tags(df):
    # Tagged as illustration
    idx_tokeep = df.index[df.tag == 'illus']
    df.loc[idx_tokeep, 'problematic'] = 'maybe'
    # Any image of grasses
    idx_todiscard = df.index[df.ancestry.str.contains('Poaceae')]
    df.loc[idx_todiscard, 'problematic'] = 'maybe'

    return df

# Remove conflicting cartoonization and classification tags
def remove_conflicting_tags(df):
    # Photographic and non-photographic image class names
    photos = 'herb|null' 
    nonphotos = 'phylo|map|illus' 
    
    # Images tagged as non-photographic by cartoonization
    idx_todiscard = df.index[df.tag_cartoon == 'non-photo']
    # Make any non-photos tagged as photo by classification 'null'
    idx_todiscard = df.index[df.loc[idx_todiscard, 'tag_imtype'].str.contains(photos)]
    df.loc[idx_todiscard, 'tag_imtype'] = 'null'
    
    # Images tagged as photographic by cartoonization
    idx_todiscard = df.index[df.tag_cartoon == 'photo']
    # If any photos tagged as non-photo by classification, make 'null'
    idx_todiscard = df.index[df.loc[idx_todiscard, 'tag_imtype'].str.contains(nonphotos)]
    df.loc[idx_todiscard, 'tag_imtype'] = 'null'

    return df

# Refine botanical image classifications (plants are rule-breakers)
def refine_botanical_images(df):
    # If images are zoological, remove any botanical classifications
    if not botanical:
        idx_todiscard = df.index[df.tag == 'herb']
        df.loc[idx_todiscard, 'tag_imtype'] = 'null'
    else:
        # Filter by less strict cartoonization & confidence thresholds
        # (Herbarium sheet mnorms fall between 'photo' and 'non-photo' thresholds
        # and had fewer false classif dets)
        idx_tokeep = df.index[(df.mnorm_pp <= mnorm_thresh2) & (df.confidence > conf_thresh2)]
        df.loc[idx_tokeep, 'tag_cartoon'] = 'non-photo'
        df.loc[idx_tokeep, 'tag_imtype'] = 'herb'

    return df

In [ ]:
#@title Post-process image classifications

# Combine tagging files for batches A-D
df, base = combine_tag_files(tags_file)
print("Model predictions for Training Attempt {}, {}:".format(TRAIN_SESS_NUM, handle_base))
print("Total Images: {}\n{}".format(len(df), df[['eolMediaURL', 'mnorm_pp', 'tag', 'confidence']].head()))

# Filter predictions using determined confidence value thresholds

# Filter by cartoonization and confidence thresholds 
df = filter_by_thresholds(df, mnorm_thresh, conf_thresh)

# Add 'problematic' tags to classes model didn't learn well
df = add_problematic_tags(df) 

# Remove conflicting cartoonization and classification tags
df = remove_conflicting_tags(df)

# Refine botanical image classifications (plants are rule-breakers)
df = refine_botanical_images(df)

# Write results to tsv
print("\nFinal tagging dataset after filtering predictions: \n", df[['eolMediaURL', 'tag_imtype', 'tag_cartoon']].head())
outfpath = cwd + base + 'final.tsv'
print("\nSaving results to: \n", outfpath)
df.to_csv(outfpath, sep='\t', index=False)

## Display classification results on images
---

In [ ]:
#@title Adjust start index and display up to 50 images with their tags

# Adjust start index using slider
start = 0 #@param {type:"slider", min:0, max:5000, step:50}
stop = min((start+50), len(df))

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[start:stop].iterrows():
    try:
        # Read in image from url
        url = df['eolMediaURL'][i]
        fn = str(i) + '.jpg'
        img, disp_img, img_cv = image_from_url(url, fn)
    
        # Get image type tag
        tag = df['tag_imtype'][i]
    
        # Display progress message after each image is loaded
        print('Successfully loaded {} of {} images'.format(i+1, (stop-start)))

        # Show classification results for images
        # Only use to view predictions on <50 images at a time
        _, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(disp_img)
        plt.axis('off')
        plt.title("{}) Image type: {} ".format(i+1, tag))

    except:
        print('Check if URL from {} is valid'.format(url))